In [38]:
import pyreadr
import numpy as np 
import lightgbm as lgb
import pandas as pd
from sklearn import preprocessing

result = pyreadr.read_r('df_train_test.RData')

population_train = result['population_train']
population_test = result['population_test']
df_train = result['df_train']
df_test = result['df_test']

# Create pandas DataFrame Object
df = pd.DataFrame(population_train)
# Create LabelEncoder Object and fit(),transform()
lbl = preprocessing.LabelEncoder()
lbl.fit(np.ravel(df))
population_train = lbl.transform(np.ravel(df))

train_data = lgb.Dataset(df_train, label=population_train)

population_test =  population_test.to_numpy()

In [54]:
params = {'objective': 'multiclass', 'metric': 'multi_error', 'num_class':5, 'force_col_wise':True,
          'verbose': -1, 
          'num_leaves': 31,'max_depth': 3, 'bagging_fraction': 0.1}

best_params = {'objective': 'multiclass', 'metric': 'multi_error', 'num_class':5, 'force_col_wise':True,
               'verbose': -1,
               'num_leaves': 31, 'max_depth': 3, 'bagging_fraction': 0.1}

min_merror = 100

In [59]:
for max_depth in range(0,6,1):
    for num_leaves in range(20, 40,5):
        for bagging_fraction in np.arange(0.1, 1,0.2):
                params['max_depth'] = max_depth
                params['bagging_fraction'] = bagging_fraction
                params['num_leaves'] = num_leaves

                cv_results = lgb.cv(
                                    params,
                                    train_data,
                                    seed=42,
                                    nfold=3
                                    )

                mean_merror = pd.Series(cv_results['multi_error-mean']).min()
                #boost_rounds = pd.Series(cv_results['multi_error-mean']).argmin()

                if mean_merror < min_merror:
                    min_merror = mean_merror
                    best_params['max_depth']= max_depth
                    best_params['bagging_fraction'] = bagging_fraction
                    best_params['num_leaves'] = num_leaves

In [60]:
bst = lgb.train(best_params, train_data)

ypred = bst.predict(df_test)

cl = np.unique(population_test)

right = 0

for i in range(1, len(ypred)):
    x = ypred[i]
    pred = cl[np.argmax(x)]
    if pred == population_test[i]:
        right = right + 1

print(f"Test accuracy = {round(right/len(population_test)*100,3)}%.")


Test accuracy = 74.834%.
